In [7]:
import urllib.request
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer

In [8]:
train_df = pd.read_excel('/Users/gimmingyeong/Documents/3-졸업작품/3-2/U-Friends/감성대화_데이터셋/Training/감성대화말뭉치(최종데이터)_Training/감성대화말뭉치(최종데이터)_Training.xlsx')

test_df = pd.read_excel('/Users/gimmingyeong/Documents/3-졸업작품/3-2/U-Friends/감성대화_데이터셋/Validation/감성대화말뭉치(최종데이터)_Validation/감성대화말뭉치(최종데이터)_Validation.xlsx')

# Preprocessing

In [9]:
print(len(train_df))
print(len(test_df))

40879
5130


In [10]:
train_df.set_index("번호", inplace=True)
test_df.set_index("번호", inplace=True)

In [11]:
train_df = train_df.drop(["연령", "성별", "상황키워드", "신체질환", "감정_소분류", 
                           "사람문장2", "시스템응답2", "사람문장3", 
                          "시스템응답3", "사람문장2", "시스템응답4",  "사람문장4"], axis=1)
test_df = test_df.drop(["연령", "성별", "상황키워드", "신체질환", "감정_소분류", 
                           "사람문장2", "시스템응답2", "사람문장3", 
                          "시스템응답3", "사람문장2", "시스템응답4",  "사람문장4"], axis=1)

In [12]:
train_df.head(5)

,감정_대분류,사람문장1,시스템응답1
번호,,,
44164,기쁨,아내가 드디어 출산하게 되어서 정말 신이 나.,아내분이 출산을 하시는군요. 정말 축하드려요.
3926,불안,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.,약 종류가 많아 번거로우시겠어요.
50882,당황,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.,고등학교 수업이 중학교 때와는 다르게 갑자기 어려워서 당황스러우시군요.
31303,기쁨,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.,재취업 후 첫 월급이라 정말 기쁘시겠어요.
47200,기쁨,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.,기분 좋으시겠어요. 앞으로는 어떻게 하실 생각이신가요?


In [13]:
# 감정_대분류 : '불안 ', '기쁨 ' -> '불안', '기쁨'
train_df.loc[train_df['감정_대분류'] == '기쁨 ', '감정_대분류'] = '기쁨'
train_df.loc[train_df['감정_대분류'] == '불안 ', '감정_대분류'] = '불안'

test_df.loc[test_df['감정_대분류'] == '기쁨 ', '감정_대분류'] = '기쁨'
test_df.loc[test_df['감정_대분류'] == '불안 ', '감정_대분류'] = '불안'

In [14]:
# document 열과 label 열의 중복을 제외한 값의 개수
train_df['사람문장1'].nunique(), train_df['감정_대분류'].nunique()

(39415, 6)

In [15]:
# 총 40,879개의 샘플이 존재하는데 '사람문장1'열에서 중복을 제거한 샘플의 개수가 39,415개라는 것은 약 6,000개의 중복 샘플이 존재한다는 의미
# '감정_대분류'열은 6이 출력됨(기쁨, 당황, 분노, 불안, 상처, 슬픔). 중복 샘플을 제거


# document 열의 중복 제거
train_df.drop_duplicates(subset=['사람문장1'], inplace=True)
test_df.drop_duplicates(subset = ['사람문장1'], inplace=True) # 사람문장1 열에서 중복인 내용이 있다면 중복 제거

In [16]:
train_df.drop_duplicates(subset=['시스템응답1'], inplace=True)
test_df.drop_duplicates(subset = ['시스템응답1'], inplace=True)

In [17]:
print('총 샘플의 수 :',len(train_df))
print('총 샘플의 수 :',len(test_df))

총 샘플의 수 : 38966
총 샘플의 수 : 5095


In [18]:
# train_df['감정_대분류'].value_counts().plot(kind = 'bar')

In [19]:
print(train_df.groupby('감정_대분류').size().reset_index(name = 'count'))
print(test_df.groupby('감정_대분류').size().reset_index(name = 'count'))

  감정_대분류  count
0     기쁨   6414
1     당황   6027
2     분노   6570
3     불안   7035
4     상처   6334
5     슬픔   6586
  감정_대분류  count
0     기쁨    838
1     당황    824
2     분노    866
3     불안    900
4     상처    813
5     슬픔    854


In [20]:
print(train_df.isnull().values.any()) # null 값을 가진 샘플이 없다는 뜻
print(test_df.isnull().values.any())

False
False


In [21]:
print(len(train_df))
print(len(test_df))

38966
5095


In [22]:
# 한글과 공백을 제외하고 모두 제거
train_df['사람문장1'] = train_df['사람문장1'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
test_df['사람문장1'] = test_df['사람문장1'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
train_df['사람문장1'] = train_df['사람문장1'].str.replace('^ +', "") # white space 데이터를 empty value로 변경
test_df['사람문장1'] = test_df['사람문장1'].str.replace('^ +', "") # 공백은 empty 값으로 변경

train_df['시스템응답1'] = train_df['시스템응답1'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
test_df['시스템응답1'] = test_df['시스템응답1'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
train_df['시스템응답1'] = train_df['시스템응답1'].str.replace('^ +', "") # white space 데이터를 empty value로 변경
test_df['시스템응답1'] = test_df['시스템응답1'].str.replace('^ +', "") # 공백은 empty 값으로 변경

train_df[:5]

/var/folders/7s/y6488qd53mj76gf6zw689dtw0000gn/T/ipykernel_8981/1170023578.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  train_df['사람문장1'] = train_df['사람문장1'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
/var/folders/7s/y6488qd53mj76gf6zw689dtw0000gn/T/ipykernel_8981/1170023578.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  test_df['사람문장1'] = test_df['사람문장1'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
/var/folders/7s/y6488qd53mj76gf6zw689dtw0000gn/T/ipykernel_8981/1170023578.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  train_df['사람문장1'] = train_df['사람문장1'].str.replace('^ +', "") # white space 데이터를 empty value로 변경
/var/folders/7s/y6488qd53mj76gf6zw689dtw0000gn/T/ipykernel_8981/1170023578.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  test_df['사람문장1'] = test_df['사람문장1'].str.replac

,감정_대분류,사람문장1,시스템응답1
번호,,,
44164,기쁨,아내가 드디어 출산하게 되어서 정말 신이 나,아내분이 출산을 하시는군요 정말 축하드려요
3926,불안,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야,약 종류가 많아 번거로우시겠어요
50882,당황,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워,고등학교 수업이 중학교 때와는 다르게 갑자기 어려워서 당황스러우시군요
31303,기쁨,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야 너무 행복해,재취업 후 첫 월급이라 정말 기쁘시겠어요
47200,기쁨,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어,기분 좋으시겠어요 앞으로는 어떻게 하실 생각이신가요


In [23]:
# 한글이 없는 리뷰였다면 더 이상 아무런 값도 없는 빈(empty) 값이 되었을 것
# 그런 샘플이 있나 확인

train_df['사람문장1'].replace('', np.nan, inplace=True)
test_df['사람문장1'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경

train_df['시스템응답1'].replace('', np.nan, inplace=True)
test_df['시스템응답1'].replace('', np.nan, inplace=True)

print(train_df.isnull().sum())
print(test_df.isnull().sum())

감정_대분류    0
사람문장1     0
시스템응답1    0
dtype: int64
감정_대분류    0
사람문장1     0
시스템응답1    0
dtype: int64


In [24]:
train_df.loc[(train_df['감정_대분류']=="분노"), '감정_대분류'] = 0
train_df.loc[(train_df['감정_대분류']=="슬픔"), '감정_대분류'] = 1
train_df.loc[(train_df['감정_대분류']=="불안"), '감정_대분류'] = 2
train_df.loc[(train_df['감정_대분류']=="상처"), '감정_대분류'] = 3
train_df.loc[(train_df['감정_대분류']=="당황"), '감정_대분류'] = 4
train_df.loc[(train_df['감정_대분류']=="기쁨"), '감정_대분류'] = 5

test_df.loc[(test_df['감정_대분류']=="분노"), '감정_대분류'] = 0
test_df.loc[(test_df['감정_대분류']=="슬픔"), '감정_대분류'] = 1
test_df.loc[(test_df['감정_대분류']=="불안"), '감정_대분류'] = 2
test_df.loc[(test_df['감정_대분류']=="상처"), '감정_대분류'] = 3
test_df.loc[(test_df['감정_대분류']=="당황"), '감정_대분류'] = 4
test_df.loc[(test_df['감정_대분류']=="기쁨"), '감정_대분류'] = 5

In [25]:
train_df.rename(columns={'감정_대분류':'sentiment', '사람문장1':'user', '시스템응답1':'chatbot'}, inplace=True)

In [26]:
train_df.head()

,sentiment,user,chatbot
번호,,,
44164,5,아내가 드디어 출산하게 되어서 정말 신이 나,아내분이 출산을 하시는군요 정말 축하드려요
3926,2,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야,약 종류가 많아 번거로우시겠어요
50882,4,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워,고등학교 수업이 중학교 때와는 다르게 갑자기 어려워서 당황스러우시군요
31303,5,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야 너무 행복해,재취업 후 첫 월급이라 정말 기쁘시겠어요
47200,5,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어,기분 좋으시겠어요 앞으로는 어떻게 하실 생각이신가요


# 챗봇 구현
우선 train data만 사용

In [27]:
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

In [28]:
train_df['embedding'] = train_df.apply(lambda row: model.encode(row.user), axis = 1)

In [ ]:
# train_df.to_csv('train_df_embedding.csv', index=False)

In [29]:
train_df

,sentiment,user,chatbot,embedding
번호,,,,
44164,5,아내가 드디어 출산하게 되어서 정말 신이 나,아내분이 출산을 하시는군요 정말 축하드려요,"[-0.49966457, -0.19717224, -1.1815602, -0.5376..."
3926,2,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야,약 종류가 많아 번거로우시겠어요,"[-0.31580722, -0.35631898, -0.22018802, 0.3595..."
50882,4,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워,고등학교 수업이 중학교 때와는 다르게 갑자기 어려워서 당황스러우시군요,"[-0.2681434, -0.07933197, 0.29177105, -0.23386..."
31303,5,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야 너무 행복해,재취업 후 첫 월급이라 정말 기쁘시겠어요,"[-0.4276825, 0.11820968, -0.29986903, 0.052228..."
47200,5,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어,기분 좋으시겠어요 앞으로는 어떻게 하실 생각이신가요,"[0.27552524, -0.2071679, -0.3845357, -0.231276..."
...,...,...,...,...
32233,3,최 영감이 노인들에게 나를 이상한 사람으로 이야기해서 답답하고 억울해,최 영감이 노인들에게 이상하게 이야기해서 억울하셨군요,"[-0.39596567, -0.42922038, 0.89251107, -0.3699..."
48959,5,지난주에 건강검진 결과가 나왔는데 정상이라고 결과가 나왔어,검진 결과가 좋게 나와서 안도하겠네요,"[-0.65557665, -0.118350714, 0.023822047, 0.221..."
36344,1,엄마는 내 꿈인 작가를 응원해 주고는 했는데 지금은 안 그래 너무 슬퍼,어머니의 응원이 사라져서 슬프시겠어요 조금 더 자세히 말해주시겠어요,"[-0.24601805, -0.048332993, -0.33113316, -0.14..."


In [30]:
from numpy import dot
from numpy.linalg import norm

from sklearn.metrics.pairwise import cosine_similarity

In [31]:
text = '요즘 머리가 아프고 너무 힘들어'
embedding = model.encode(text)

In [32]:
train_df['similarity'] = train_df['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())

In [33]:
train_df.head()

,sentiment,user,chatbot,embedding,similarity
번호,,,,,
44164,5,아내가 드디어 출산하게 되어서 정말 신이 나,아내분이 출산을 하시는군요 정말 축하드려요,"[-0.49966457, -0.19717224, -1.1815602, -0.5376...",0.201964
3926,2,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야,약 종류가 많아 번거로우시겠어요,"[-0.31580722, -0.35631898, -0.22018802, 0.3595...",0.544759
50882,4,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워,고등학교 수업이 중학교 때와는 다르게 갑자기 어려워서 당황스러우시군요,"[-0.2681434, -0.07933197, 0.29177105, -0.23386...",0.394498
31303,5,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야 너무 행복해,재취업 후 첫 월급이라 정말 기쁘시겠어요,"[-0.4276825, 0.11820968, -0.29986903, 0.052228...",0.214048
47200,5,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어,기분 좋으시겠어요 앞으로는 어떻게 하실 생각이신가요,"[0.27552524, -0.2071679, -0.3845357, -0.231276...",0.203653


In [34]:
answer = train_df.loc[train_df['similarity'].idxmax()]

print('구분: ', answer['sentiment'])
print('유사한 질문: ', answer['user'])
print('챗봇 답변: ', answer['chatbot'])
print('유사도: ', answer['similarity']) # 1에 가까울수록 유사

구분:  2
유사한 질문:  요즘에 머리가 너무 아파서 걱정이야
챗봇 답변:  요즘에 머리가 너무 아파서 걱정이시군요무슨 일 있으신가요
유사도:  0.9230511784553528
